# **Hand gesture Classification using SVM**
Submitted By:  
> **Group 19**  
*18EE30021 : Pratyush Jaiswal*    
*18EE30029 : Nuruddin Jiruwala*       
*18EE10043 : Rhitvik Sinha*  
*18EE10062 : Akshat Kumar Gupta*  




In [ ]:
import cv2
import os
import numpy as np
import pandas as pd
import random
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
La=os.listdir("A/")
Ln=os.listdir("nothing/")
#listdir gives list in alphabetical order which is harmful for us so it is ditched later on

In [ ]:
#Trying canny edge detection for one image
img=cv2.imread("A/"+La[0],0)

imgb=cv2.imread("nothing/"+Ln[0],0)

finala=img-imgb
#_,finale=cv2.threshold(finala,60,255,cv2.THRESH_BINARY)
final_small=cv2.resize(finala,(50,50),interpolation=cv2.INTER_AREA)

edges=cv2.Canny(final_small,400,700)

cv2.imshow('image',edges)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
edges.shape

(50, 50)

 ## Data preprocessing

We have to predict what the hand gesture of any image could be. So one approach could be to find the edge around the hand as best as we can and those pixels could give us the relevant info and be our features respectively. Here 9 classes A-I are there with 3000 images each. So total 3000 X 9 i.e. 27000 images. 70% gives 18900 images. This data set is huge and SVM would never compile as its time complexity depends on the square of the number of datapoints. So took 300(train+test) images instead of all. Also the image dimensions are 200X200, this is scaled down to 50X50 as each pixel would be a feature here. Canny is used for edge detection on image-background.

In [ ]:
labels=['A','B','C','D','E','F','G','H','I']
train_features=[]
train_labels=[]
test_features=[]
test_labels=[]


In [ ]:
for i in range(len(labels)):
    l=list(range(300))
    train_pics_indices=random.sample(l,210) # 10% of original dataset
    test_pics_indices=list(np.setdiff1d(l,train_pics_indices)) # get remaining 90 indices
    
    for idx in train_pics_indices:
        img_name=labels[i]+str(idx+1)+'.jpg'
        img_og=cv2.imread(labels[i]+'/'+img_name,0) # reading in B and W
        img_bg='nothing'+str(idx+1)+'.jpg'
        img_bg=cv2.imread('nothing'+'/'+img_bg,0)
        img=img_og-img_bg
        final_img=cv2.resize(img,(50,50),interpolation=cv2.INTER_AREA)
        edgy=cv2.Canny(final_img,400,700) # params were set through trial and error and some internet searching
        edgy=np.array(edgy==255,dtype='int32') # change to 0 1 binary
        edgy=list(np.reshape(edgy,(-1,))) # change to 1D
        train_features.append(edgy)
        train_labels.append(i)
        
    # doing the same for test data set
    for idx in test_pics_indices:
        img_name=labels[i]+str(idx+1)+'.jpg'
        img_og=cv2.imread(labels[i]+'/'+img_name,0) # reading in B and W
        img_bg='nothing'+str(idx+1)+'.jpg'
        img_bg=cv2.imread('nothing'+'/'+img_bg,0)
        img=img_og-img_bg
        final_img=cv2.resize(img,(50,50),interpolation=cv2.INTER_AREA)
        edgy=cv2.Canny(final_img,400,700) # params were set through trial and error and some internet searching
        edgy=np.array(edgy==255,dtype='int32') # change to 0 1 binary
        edgy=list(np.reshape(edgy,(-1,))) # change to 1D
        test_features.append(edgy)
        test_labels.append(i)
        
        

In [ ]:
np.array(train_features).shape

(1890, 2500)

In [ ]:
np.array(train_labels).shape

(1890,)

In [ ]:
np.array(test_features).shape

(810, 2500)

In [ ]:
np.array(test_labels).shape

(810,)

In [ ]:
for i in range(1890):
    train_features[i].append(train_labels[i])  # append features and label for final dataframe
for i in range(810):
    test_features[i].append(test_labels[i])

In [ ]:
cols=[]
for i in range(2500):
    cols.append('Feat_'+str(i+1))
cols.append('Label')

In [ ]:
df_train=pd.DataFrame(train_features,columns=cols)
df_train = df_train.sample(frac = 1)
df_train.reset_index(drop=True)

,Feat_1,Feat_2,Feat_3,Feat_4,Feat_5,Feat_6,Feat_7,Feat_8,Feat_9,Feat_10,...,Feat_2492,Feat_2493,Feat_2494,Feat_2495,Feat_2496,Feat_2497,Feat_2498,Feat_2499,Feat_2500,Label
0,0,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
1,0,0,0,1,1,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,2
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
3,0,0,0,0,1,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,2
4,0,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1885,0,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1886,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,2
1887,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,4
1888,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
df_test=pd.DataFrame(test_features,columns=cols)
df_test = df_test.sample(frac = 1)
df_test.reset_index(drop=True)

,Feat_1,Feat_2,Feat_3,Feat_4,Feat_5,Feat_6,Feat_7,Feat_8,Feat_9,Feat_10,...,Feat_2492,Feat_2493,Feat_2494,Feat_2495,Feat_2496,Feat_2497,Feat_2498,Feat_2499,Feat_2500,Label
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,7
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
805,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
806,0,0,0,1,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
807,0,1,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
808,0,0,1,1,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,5


## CSV file making

In [ ]:
df_train.to_csv('Train.csv')
df_test.to_csv('Test.csv')

## Running SVM

In [ ]:
X_train=df_train.drop(['Label'],axis=1).values
X_test=df_test.drop(['Label'],axis=1).values
Y_train=df_train['Label'].values
Y_test=df_test['Label'].values

In [ ]:
vanilla_model=SVC()
vanilla_model.fit(X_train,Y_train) # fitting default SVM for training set

SVC()

In [ ]:
Y_pred=vanilla_model.predict(X_test)
acc=accuracy_score(Y_pred,Y_test)

In [ ]:
acc

0.991358024691358

In [ ]:
print(classification_report(Y_test,Y_pred))

              precision    recall  f1-score   support

           0       1.00      0.98      0.99        90
           1       0.98      0.99      0.98        90
           2       1.00      1.00      1.00        90
           3       0.99      1.00      0.99        90
           4       0.97      0.98      0.97        90
           5       1.00      0.98      0.99        90
           6       0.99      1.00      0.99        90
           7       1.00      1.00      1.00        90
           8       1.00      1.00      1.00        90

    accuracy                           0.99       810
   macro avg       0.99      0.99      0.99       810
weighted avg       0.99      0.99      0.99       810



## Hyperparamter tuning using grid search

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = {'kernel': ['rbf','linear','polynomial']}       # changing all hyperparameters was taking extremely long, so changing one at a time
grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3) # refit does a train-validation split and again retrains on full train for best output
grid.fit(X_train, Y_train)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
[CV] kernel=rbf ......................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] .......................... kernel=rbf, score=0.966, total=  17.3s
[CV] kernel=rbf ......................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   17.2s remaining:    0.0s


[CV] .......................... kernel=rbf, score=0.995, total=  16.3s
[CV] kernel=rbf ......................................................


...

C:\Users\Master\anaconda3\envs\py3env\lib\site-packages\sklearn\model_selection\_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Master\anaconda3\envs\py3env\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Master\anaconda3\envs\py3env\lib\site-packages\sklearn\svm\_base.py", line 217, in fit
    fit(X, y, sample_weight, solver_type, kernel, random_seed=seed)
  File "C:\Users\Master\anaconda3\envs\py3env\lib\site-packages\sklearn\svm\_base.py", line 268, in _dense_fit
    self._probB, self.fit_status_ = libsvm.fit(
  File "sklearn\svm\_libsvm.pyx", line 175, in sklearn.svm._libsvm.fit
ValueError: 'polynomial' is not in list

  warnings.warn("Estimator fit failed. The score on this train-test"
C:\Users\Master\anaconda3\envs\py3env\li

[CV] ..................... kernel=polynomial, score=nan, total=   0.1s
[CV] kernel=polynomial ...............................................
[CV] ..................... kernel=polynomial, score=nan, total=   0.1s
[CV] kernel=polynomial ...............................................


C:\Users\Master\anaconda3\envs\py3env\lib\site-packages\sklearn\model_selection\_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Master\anaconda3\envs\py3env\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Master\anaconda3\envs\py3env\lib\site-packages\sklearn\svm\_base.py", line 217, in fit
    fit(X, y, sample_weight, solver_type, kernel, random_seed=seed)
  File "C:\Users\Master\anaconda3\envs\py3env\lib\site-packages\sklearn\svm\_base.py", line 268, in _dense_fit
    self._probB, self.fit_status_ = libsvm.fit(
  File "sklearn\svm\_libsvm.pyx", line 175, in sklearn.svm._libsvm.fit
ValueError: 'polynomial' is not in list

  warnings.warn("Estimator fit failed. The score on this train-test"
[Parallel(n_jobs=1)]: Done  15 out of  1

[CV] ..................... kernel=polynomial, score=nan, total=   0.1s


GridSearchCV(estimator=SVC(),
             param_grid={'kernel': ['rbf', 'linear', 'polynomial']}, verbose=3)

In [ ]:
print(grid.best_params_)

{'kernel': 'rbf'}


In [ ]:
param_grid = {'kernel': ['rbf'],'C': [0.1, 1, 10, 100, 1000]}       # changing all hyperparameters was taking extremely long, so changing one at a time
grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3) # refit does a train-validation split and again retrains on full train for best output
grid.fit(X_train, Y_train)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV] C=0.1, kernel=rbf ...............................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ................... C=0.1, kernel=rbf, score=0.757, total=  18.3s
[CV] C=0.1, kernel=rbf ...............................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   18.2s remaining:    0.0s


[CV] ................... C=0.1, kernel=rbf, score=0.788, total=  20.5s
[CV] C=0.1, kernel=rbf ...............................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   38.7s remaining:    0.0s


[CV] ................... C=0.1, kernel=rbf, score=0.759, total=  22.2s
[CV] C=0.1, kernel=rbf ...............................................
[CV] ................... C=0.1, kernel=rbf, score=0.757, total=  23.1s
[CV] C=0.1, kernel=rbf ...............................................
[CV] ................... C=0.1, kernel=rbf, score=0.786, total=  23.1s
[CV] C=1, kernel=rbf .................................................
[CV] ..................... C=1, kernel=rbf, score=0.966, total=  22.3s
[CV] C=1, kernel=rbf .................................................
[CV] ..................... C=1, kernel=rbf, score=0.995, total=  22.6s
[CV] C=1, kernel=rbf .................................................
[CV] ..................... C=1, kernel=rbf, score=0.992, total=  22.4s
[CV] C=1, kernel=rbf .................................................
[CV] ..................... C=1, kernel=rbf, score=0.979, total=  22.4s
[CV] C=1, kernel=rbf .................................................
[CV] .

[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:  9.3min finished


GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100, 1000], 'kernel': ['rbf']},
             verbose=3)

In [ ]:
print(grid.best_params_)

{'C': 10, 'kernel': 'rbf'}


In [ ]:
param_grid = {'kernel': ['rbf'],'C': [10],'gamma': [1, 0.1, 0.01, 0.001, 0.0001]}       # changing all hyperparameters was taking extremely long, so changing one at a time
grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3) # refit does a train-validation split and again retrains on full train for best output
grid.fit(X_train, Y_train)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV] C=10, gamma=1, kernel=rbf .......................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ........... C=10, gamma=1, kernel=rbf, score=0.135, total=  17.4s
[CV] C=10, gamma=1, kernel=rbf .......................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   17.3s remaining:    0.0s


[CV] ........... C=10, gamma=1, kernel=rbf, score=0.127, total=  17.1s
[CV] C=10, gamma=1, kernel=rbf .......................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   34.3s remaining:    0.0s


[CV] ........... C=10, gamma=1, kernel=rbf, score=0.119, total=  17.1s
[CV] C=10, gamma=1, kernel=rbf .......................................
[CV] ........... C=10, gamma=1, kernel=rbf, score=0.130, total=  17.1s
[CV] C=10, gamma=1, kernel=rbf .......................................
[CV] ........... C=10, gamma=1, kernel=rbf, score=0.130, total=  16.9s
[CV] C=10, gamma=0.1, kernel=rbf .....................................
[CV] ......... C=10, gamma=0.1, kernel=rbf, score=0.349, total=  16.8s
[CV] C=10, gamma=0.1, kernel=rbf .....................................
[CV] ......... C=10, gamma=0.1, kernel=rbf, score=0.370, total=  16.9s
[CV] C=10, gamma=0.1, kernel=rbf .....................................
[CV] ......... C=10, gamma=0.1, kernel=rbf, score=0.310, total=  16.9s
[CV] C=10, gamma=0.1, kernel=rbf .....................................
[CV] ......... C=10, gamma=0.1, kernel=rbf, score=0.339, total=  17.2s
[CV] C=10, gamma=0.1, kernel=rbf .....................................
[CV] .

[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:  7.8min finished


GridSearchCV(estimator=SVC(),
             param_grid={'C': [10], 'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf']},
             verbose=3)

In [ ]:
print(grid.best_params_)

{'C': 10, 'gamma': 0.01, 'kernel': 'rbf'}


In [ ]:
grid_Y_test=grid.predict(X_test)

In [ ]:
print(classification_report(Y_test,grid_Y_test))

              precision    recall  f1-score   support

           0       1.00      0.98      0.99        90
           1       1.00      1.00      1.00        90
           2       1.00      1.00      1.00        90
           3       1.00      1.00      1.00        90
           4       0.98      1.00      0.99        90
           5       1.00      0.99      0.99        90
           6       0.99      1.00      0.99        90
           7       1.00      1.00      1.00        90
           8       1.00      1.00      1.00        90

    accuracy                           1.00       810
   macro avg       1.00      1.00      1.00       810
weighted avg       1.00      1.00      1.00       810



In [ ]:
print(accuracy_score(Y_test,grid_Y_test))

0.9962962962962963
